# MMagic Inference Tutorial

Welcome to MMagic! This is the official tutorial for using MMagic inference api to predict your own image or video.

In this tutorial, you will learn how to

[1. Install MMagic](#1-install-mmagic)

[2. Check inference supported tasks and models](#2-check-inference-supported-tasks-and-models)

[3. Perform inference using MMagic
 inference API](#3-perform-inference-with-mmagic-api)

&emsp; [3.1 Prepare some images or videos for inference](#31-prepare-some-images-or-videos-for-inference)

&emsp; [3.2 Perform inference with two lines of python code](#32-perform-inference-with-two-lines-of-python-code)

&emsp; [3.3 Infer with different settings of a specific model](#33-infer-with-different-settings-of-a-specific-model)

&emsp; [3.4 Infer with extra parameters](#34-inference-with-extra-parameters)

[4. Perform inference with models of different tasks including](#4-perform-inference-with-models-of-different-tasks):

&emsp; [4.1 Inference of conditional GANs models](#41-inference-of-conditional-gan-models)

&emsp; [4.2 Inference of inpanting models](#42-inference-of-inpainting-models)

&emsp; [4.3 Inference of matting models](#43-inference-of-matting-models)

&emsp; [4.4 Inference of super resolution models](#44-inference-of-image-super-resolution-models)

&emsp; [4.5 Inference of image2image models](#45-inference-of-image-translation-models)

&emsp; [4.6 Inference of unconditional GANs models](#46-inference-of-unconditional-gan-models)

&emsp; [4.7 Inference of video interpolation models](#47-inference-of-video-interpolation-models)

&emsp; [4.8 Inference of video super resolution models](#48-inference-of-video-restoration-models)

&emsp; [4.9 Inference of text-to-image models](#49-inference-of-text-to-image-models)

Let's start!

## 1. Install MMagic

In [ ]:
# Check PyTorch version
!pip3 list | grep torch

In [ ]:
# Install mmcv dependency via openmim
!pip3 install openmim
!mim install 'mmcv>=2.0.0'

In [ ]:
# Install mmagic from source
%cd ..
!pip3 install -e .
%cd demo

In [ ]:
# Check MMagic installation
import mmagic
print(mmagic.__version__)

## 2. Check inference supported tasks and models

There are multiple task types in MMagic: Matting, Inpainting, Video Super-Resolution, Image Super-Resolution, Image2Image, Unconditional GANs, Conditional GANs, Video Interpolation. 

We provide some models for each task. All available models and tasks could be printed out like this.

In [ ]:
from mmagic.apis import MMagicInferencer

# print all supported models for inference.
inference_supported_models = MMagicInferencer.get_inference_supported_models()
print('all supported models:')
print(inference_supported_models)

# print all supported tasks for inference.
supported_tasks = MMagicInferencer.get_inference_supported_tasks()
print('all supported tasks:')
print(supported_tasks)

# print all supported models for one task, take image translation for example.
task_supported_models = MMagicInferencer.get_task_supported_models('Image2Image')
print('translation models:')
print(task_supported_models)

## 3. Perform inference with MMagic API

Next we describe how to perform inference with python code snippets.

(We also provide command line interface for you to do inference by running mmagic_inference_demo.py. The usage of this interface could be found in [README.md](./README.md) and more guidance could be found in the [documentation](https://mmagic.readthedocs.io/en/latest/user_guides/3_inference.html#).)


### 3.1 Prepare some images or videos for inference

Before we start to perform inference with a pretrained model, some input images or videos should be prepared. 

Take image translation for example. We need a input image to be translated.

Put your image to some directory and make a directory to save processed image.

In [ ]:
# make a dir for input image and output image
!mkdir -p ./../resources/input/translation
!mkdir -p ./../resources/output/translation

We also have prepared some images and videos for you. You can download by running [download_inference_resouces.py](./download_inference_resources.py).

This script allows you to see what resources are available and makes download easier.

In [ ]:
# see all resources
!python download_inference_resources.py --print-all
# see all task types
!python download_inference_resources.py --print-task-type
# see resources of one specific task
!python download_inference_resources.py --print-task 'Inpainting'
# download all resouces to default dir '../resources'
!python download_inference_resources.py
# download resouces of one task
!python download_inference_resources.py --task 'Inpainting'
# download to the directory you want
!python download_inference_resources.py --root-dir '../resources'

### 3.2 Perform inference with two lines of python code. 

There are two steps:

First, create a MMagicInferencer instance by a pretrained model name.

Second, infer your own image with this MMagicInferencer instance. The translated image will be saved to result_out_dir.

In [ ]:
from mmagic.apis import MMagicInferencer

# Create a MMagicInferencer instance
editor = MMagicInferencer('pix2pix')
# Infer a image. Input image path and output image path is needed.
results = editor.infer(img='../resources/input/translation/gt_mask_0.png', result_out_dir='../resources/output/translation/tutorial_translation_pix2pix_res.jpg')

You could see your result image by plotting it out.

In [ ]:
# plot the result image
import mmcv
import matplotlib.pyplot as plt 
img = mmcv.imread('../resources/output/translation/tutorial_translation_pix2pix_res.jpg')
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

### 3.3 Infer with different settings of a specific model

There are some different configs and checkpoints for one model.

Take conditional GAN model 'biggan' as an example. We have pretrained model for Cifar and Imagenet, and all pretrained models of 'biggan' are listed in its [metafile.yaml](../configs/biggan/metafile.yml)

You could configure different settings by passing 'model_setting' to 'MMagicInferencer'. Every model's default setting is 0.

In [ ]:
import mmcv
import matplotlib.pyplot as plt
from mmagic.apis import MMagicInferencer

result_out_dir = '../resources/output/conditional/tutorial_conditinal_biggan_res_setting1.jpg'
# configure setting to 1
editor = MMagicInferencer('biggan', model_setting=1) 
results = editor.infer(label=1, result_out_dir=result_out_dir)

# plot the result image
img = mmcv.imread(result_out_dir)
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

### 3.4 Infer with extra parameters

Some models may have extra parameters that could be configured to perform inference.

Take 'biggan' for example. You could configure 'num_batches' in a dict and pass it to 'MMagicInferencer'.

In [ ]:
import mmcv
import matplotlib.pyplot as plt
from mmagic.apis import MMagicInferencer

result_out_dir = '../resources/output/conditional/tutorial_conditinal_biggan_res_sample6.jpg'
# use a dict to pass the parameters, num_batches means images output num for one inference
editor = MMagicInferencer('biggan', model_setting=1, extra_parameters={'num_batches':6}) 
results = editor.infer(label=1, result_out_dir=result_out_dir)

# plot the result image and we could see 6 images in a inference batch
img = mmcv.imread(result_out_dir)
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

To know what extra parameters that a model have, do like this.

In [ ]:
from mmagic.apis import MMagicInferencer

editor = MMagicInferencer('biggan', model_setting=1) 
editor.print_extra_parameters()
# 'num_batches' and 'sample_model' are extra parameters in 'biggan' model.

## 4. Perform inference with models of different tasks

### 4.1 Inference of conditional GAN models

Conditional GAN models take a label as input and output a image. We take 'biggan' as an example.

In [ ]:
import mmcv
import matplotlib.pyplot as plt 
from mmagic.apis import MMagicInferencer

# Create a MMagicInferencer instance and infer
result_out_dir = '../resources/output/conditional/tutorial_conditinal_biggan_res.jpg'
editor = MMagicInferencer('biggan', model_setting=1)
results = editor.infer(label=1, result_out_dir=result_out_dir)

# plot the result image
img = mmcv.imread(result_out_dir)
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

### 4.2 Inference of inpainting models

Inpaiting models take a masked image and mask pair as input, and output a inpainted image. We take 'global_local' as an example.

In [ ]:
import mmcv
import matplotlib.pyplot as plt 
from mmagic.apis import MMagicInferencer

img = '../resources/input/inpainting/celeba_test.png'
mask = '../resources/input/inpainting/bbox_mask.png'

# show input image and mask
input_img = mmcv.imread(img)
plt.imshow(mmcv.bgr2rgb(input_img))
plt.show()
input_mask = mmcv.imread(mask)
plt.imshow(mmcv.bgr2rgb(input_mask))
plt.show()

# Create a MMagicInferencer instance and infer
result_out_dir = '../resources/output/inpainting/tutorial_inpainting_global_local_res.jpg'
editor = MMagicInferencer('global_local', model_setting=1)
results = editor.infer(img=img, mask=mask, result_out_dir=result_out_dir)

# plot the result image
img = mmcv.imread(result_out_dir)
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

### 4.3 Inference of matting models

Inpaiting models take a image and trimap pair as input, and output a alpha image. We take 'gca' as an example.

In [ ]:
import mmcv
import matplotlib.pyplot as plt 
from mmagic.apis import MMagicInferencer

img = '../resources/input/matting/GT05.jpg'
trimap = '../resources/input/matting/GT05_trimap.jpg'

# show input image and mask
input_img = mmcv.imread(img)
plt.imshow(mmcv.bgr2rgb(input_img))
plt.show()
input_trimap = mmcv.imread(trimap)
plt.imshow(mmcv.bgr2rgb(input_trimap))
plt.show()

# Create a MMagicInferencer instance and infer
result_out_dir = '../resources/output/matting/tutorial_matting_gca_res.png'
editor = MMagicInferencer('gca')
results = editor.infer(img=img, trimap=trimap, result_out_dir=result_out_dir)

# plot the result image
img = mmcv.imread(result_out_dir)
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

### 4.4 Inference of image super resolution models

Image super resolution models take a image as input, and output a high resolution image. We take 'esrgan' as an example.

In [ ]:
import mmcv
import matplotlib.pyplot as plt 
from mmagic.apis import MMagicInferencer

# Create a MMagicInferencer instance and infer
img = '../resources/input/restoration/0901x2.png'
result_out_dir = '../resources/output/restoration/tutorial_restoration_esrgan_res.png'
editor = MMagicInferencer('esrgan')
results = editor.infer(img=img, result_out_dir=result_out_dir)

# plot the result image
img = mmcv.imread(result_out_dir)
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

### 4.5 Inference of image translation models

Image translation models take a image as input, and output a translated image. We take 'pix2pix' as an example.

In [ ]:
import mmcv
import matplotlib.pyplot as plt 
from mmagic.apis import MMagicInferencer

img = '../resources/input/translation/gt_mask_0.png'

# show input image and mask
input_img = mmcv.imread(img)
plt.imshow(mmcv.bgr2rgb(input_img))
plt.show()

# Create a MMagicInferencer instance and infer
result_out_dir = '../resources/output/translation/tutorial_translation_pix2pix_res.png'
editor = MMagicInferencer('pix2pix')
results = editor.infer(img=img, result_out_dir=result_out_dir)

# plot the result image
img = mmcv.imread(result_out_dir)
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

### 4.6 Inference of unconditional GAN models

Unconditional GAN models do not need input, and output a image. We take 'styleganv1' as an example.

In [ ]:
import mmcv
import matplotlib.pyplot as plt 
from mmagic.apis import MMagicInferencer

# Create a MMagicInferencer instance and infer
result_out_dir = '../resources/output/unconditional/tutorial_unconditional_styleganv1_res.png'
editor = MMagicInferencer('styleganv2')
results = editor.infer(result_out_dir=result_out_dir)

# plot the result image
img = mmcv.imread(result_out_dir)
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

### 4.7 Inference of video interpolation models

Video interpolation models take a video as input, and output a interpolated video. We take 'flavr' as an example.

In [ ]:
import os
from mmagic.apis import MMagicInferencer
from mmengine import mkdir_or_exist

# Create a MMagicInferencer instance and infer
video = '../resources/input/video_interpolation/b-3LLDhc4EU_000000_000010.mp4'
result_out_dir = '../resources/output/video_interpolation/tutorial_video_interpolation_flavr_res.mp4'
mkdir_or_exist(os.path.dirname(result_out_dir))
editor = MMagicInferencer('flavr')
results = editor.infer(video=video, result_out_dir=result_out_dir)

Please check the result video in the output directory.

### 4.8 Inference of video restoration models

Video restoration models take a video as input, and output a restorated video. We take 'basicvsr' as an example..

In [ ]:
import os
from mmagic.apis import MMagicInferencer
from mmengine import mkdir_or_exist

# Create a MMagicInferencer instance and infer
video = '../resources/input/video_restoration/QUuC4vJs_000084_000094_400x320.mp4'
result_out_dir = '../resources/output/video_restoration/tutorial_video_restoration_edvr_res.mp4'
mkdir_or_exist(os.path.dirname(result_out_dir))
editor = MMagicInferencer('edvr', extra_parameters={'window_size':5})
results = editor.infer(video=video, result_out_dir=result_out_dir)

Please check the result video in the output directory.

### 4.9 Inference of text-to-image models

Text-to-image models take text as input, and output a image. We take 'stable_diffusion' as an example.

In [ ]:
import mmcv
import matplotlib.pyplot as plt 
from mmagic.apis import MMagicInferencer

# Create a MMagicInferencer instance and infer
editor = MMagicInferencer(model_name='stable_diffusion')
text_prompts = 'A panda is having dinner at KFC'
result_out_dir = '../resources/output/text2image/tutorial_text2image_sd_res.png'
editor.infer(text=text_prompts, result_out_dir=result_out_dir)

# plot the result image
img = mmcv.imread(result_out_dir)
plt.imshow(mmcv.bgr2rgb(img))
plt.show()